<a href="https://colab.research.google.com/github/SlowPoke228/Lessons/blob/main/L4(Tree).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

from google.colab import drive
drive.mount('/content/drive')
# url db: https://www.kaggle.com/christianlillelund/csgo-round-winner-classification
# url lesson: http://slides.com/romvano/lab4
# https://colab.research.google.com/drive/1lob7X-2Cg2zDxobcOpeI7WjCsfSh8AmA#scrollTo=xe973iwk2POD
# https://colab.research.google.com/drive/1MsWimLckvkUunfErRUs0RDHbF6Em4-_z




names = ['time_left','ct_score','t_score','map','bomb_planted','ct_health','t_health','ct_armor','t_armor','ct_money','t_money','ct_helmets',
        't_helmets','ct_defuse_kits','ct_players_alive','t_players_alive','ct_weapon_ak47','t_weapon_ak47','ct_weapon_aug','t_weapon_aug','ct_weapon_awp'
        ,'t_weapon_awp','ct_weapon_bizon','t_weapon_bizon','ct_weapon_cz75auto','t_weapon_cz75auto','ct_weapon_elite','t_weapon_elite','ct_weapon_famas',
        't_weapon_famas','ct_weapon_g3sg1','t_weapon_g3sg1','ct_weapon_galilar','t_weapon_galilar','ct_weapon_glock','t_weapon_glock','ct_weapon_m249','t_weapon_m249'
        ,'ct_weapon_m4a1s','t_weapon_m4a1s','ct_weapon_m4a4','t_weapon_m4a4','ct_weapon_mac10','t_weapon_mac10','ct_weapon_mag7'
         ,'t_weapon_mag7','ct_weapon_mp5sd','t_weapon_mp5sd','ct_weapon_mp7','t_weapon_mp7','ct_weapon_mp9','t_weapon_mp9','ct_weapon_negev','t_weapon_negev','ct_weapon_nova',
        't_weapon_nova','ct_weapon_p90','t_weapon_p90','ct_weapon_r8revolver','t_weapon_r8revolver','ct_weapon_sawedoff','t_weapon_sawedoff','ct_weapon_scar20',
       't_weapon_scar20','ct_weapon_sg553','t_weapon_sg553','ct_weapon_ssg08','t_weapon_ssg08','ct_weapon_ump45','t_weapon_ump45','ct_weapon_xm1014',
       't_weapon_xm1014','ct_weapon_deagle','t_weapon_deagle','ct_weapon_fiveseven','t_weapon_fiveseven','ct_weapon_usps','t_weapon_usps','ct_weapon_p250','t_weapon_p250',
       'ct_weapon_p2000','t_weapon_p2000','ct_weapon_tec9','t_weapon_tec9','ct_grenade_hegrenade','t_grenade_hegrenade','ct_grenade_flashbang','t_grenade_flashbang',
       'ct_grenade_smokegrenade','t_grenade_smokegrenade','ct_grenade_incendiarygrenade','t_grenade_incendiarygrenade','ct_grenade_molotovgrenade','t_grenade_molotovgrenade',
       'ct_grenade_decoygrenade','t_grenade_decoygrenade','round_winner'
]

# Подключаем бд, удаляем первую стоку и указываем тайтлы
dataset = pd.read_csv("/content/drive/MyDrive/csgo_round_snapshots.csv", names=names, skiprows=1)
#удаляем данные с слишком высокой и слишком низкой корреляцией
dataset = dataset.drop(['map','bomb_planted','ct_weapon_bizon','t_weapon_bizon','ct_weapon_cz75auto','t_weapon_cz75auto','ct_weapon_elite','t_weapon_elite','ct_weapon_famas',
        't_weapon_famas','ct_weapon_g3sg1','t_weapon_g3sg1','ct_weapon_galilar','t_weapon_galilar','ct_weapon_glock','ct_weapon_m249','t_weapon_m249'
        ,'ct_weapon_m4a1s','t_weapon_m4a1s','ct_weapon_mac10','ct_weapon_mag7','t_weapon_mag7','ct_weapon_mp5sd','t_weapon_mp5sd','ct_weapon_mp7','t_weapon_mp7','t_weapon_mp9',
        'ct_weapon_negev','t_weapon_negev','ct_weapon_nova','ct_weapon_xm1014','t_weapon_xm1014','t_weapon_nova','ct_weapon_p90','t_weapon_p90','ct_weapon_r8revolver','t_weapon_r8revolver',
        'ct_weapon_sawedoff','t_weapon_sawedoff','ct_weapon_scar20','t_weapon_scar20','ct_grenade_decoygrenade','t_grenade_decoygrenade','t_grenade_incendiarygrenade','ct_weapon_p2000','t_weapon_p2000',
        'ct_weapon_tec9','t_weapon_tec9','t_weapon_aug','ct_players_alive','t_players_alive','t_weapon_m4a4','t_weapon_ssg08','ct_grenade_molotovgrenade','ct_weapon_fiveseven','t_weapon_fiveseven'], 
        axis='columns')

dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,time_left,ct_score,t_score,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_weapon_ak47,t_weapon_ak47,ct_weapon_aug,ct_weapon_awp,t_weapon_awp,t_weapon_glock,ct_weapon_m4a4,t_weapon_mac10,ct_weapon_mp9,ct_weapon_sg553,t_weapon_sg553,ct_weapon_ssg08,ct_weapon_ump45,t_weapon_ump45,ct_weapon_deagle,t_weapon_deagle,ct_weapon_usps,t_weapon_usps,ct_weapon_p250,t_weapon_p250,ct_grenade_hegrenade,t_grenade_hegrenade,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_molotovgrenade,round_winner
0,175.00,0.0,0.0,500.0,500.0,0.0,0.0,4000.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,500.0,500.0,400.0,300.0,600.0,650.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,CT
2,96.03,0.0,0.0,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,CT
3,76.03,0.0,0.0,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,500.0,500.0,192.0,0.0,18350.0,10750.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,200.0,242.0,195.0,359.0,100.0,5950.0,2.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,T
122406,174.93,11.0,15.0,500.0,500.0,95.0,175.0,11500.0,23900.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,T
122407,114.93,11.0,15.0,500.0,500.0,495.0,475.0,1200.0,6700.0,3.0,5.0,1.0,0.0,2.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,4.0,4.0,3.0,5.0,1.0,5.0,T
122408,94.93,11.0,15.0,500.0,500.0,495.0,475.0,1200.0,6700.0,3.0,5.0,1.0,0.0,2.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,5.0,0.0,3.0,0.0,4.0,T


In [64]:
# Создаем массивы с атрибутами и метками
X = dataset.iloc[:, :-1].values  
y_temp = dataset.iloc[:, len(dataset.columns)-1].values
y=[]
#from sklearn.preprocessing import MinMaxScaler
#scaler=MinMaxScaler()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [65]:

#y=pd.DataFrame
for i in range(0,len(y_temp)):
  if y_temp[i]=='T':
    y.append(1.0)
  else:
    y.append(-1.0)


Проверка стабильности одиночного дерева

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40,random_state=42)
regressor = DecisionTreeRegressor(random_state=42).fit(X_train, y_train)
print('Точность без уменьшения признаков',regressor.score(X_test, y_test))
X_train, X_test, y_train, y_test = train_test_split(X[:, 10:], y, test_size=0.40,random_state=42)
regressor = DecisionTreeRegressor(random_state=42).fit(X_train, y_train)
print('Точность без первых 11 признаков',regressor.score(X_test, y_test))
X_train, X_test, y_train, y_test = train_test_split(X[:, :-10], y, test_size=0.40,random_state=42)
regressor = DecisionTreeRegressor(random_state=42).fit(X_train, y_train)
print('Точность без последних 11 признаков',regressor.score(X_test, y_test))

Точность без уменьшения признаков 0.16785792331751148
Точность без первых 11 признаков 0.05029846168329566
Точность без последних 11 признаков 0.2154881564213279


Отбор признаков случайным лесом

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40,random_state=42,stratify=y)
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
feature_imp = pd.Series(clf.feature_importances_,index=dataset.iloc[:, :-1].columns).sort_values(ascending=False)
feature_imp

ct_armor                        0.099856
t_armor                         0.094393
t_money                         0.064440
ct_money                        0.061998
t_helmets                       0.053326
time_left                       0.050162
t_health                        0.040138
ct_health                       0.038213
t_score                         0.037061
ct_score                        0.036565
ct_helmets                      0.034194
ct_defuse_kits                  0.032375
ct_grenade_flashbang            0.029706
t_weapon_ak47                   0.025482
ct_weapon_usps                  0.024581
t_grenade_flashbang             0.022213
t_weapon_glock                  0.021329
ct_weapon_m4a4                  0.019803
ct_grenade_smokegrenade         0.019418
t_grenade_smokegrenade          0.017639
t_weapon_sg553                  0.017420
t_grenade_molotovgrenade        0.017121
ct_grenade_hegrenade            0.015340
ct_grenade_incendiarygrenade    0.013708
ct_weapon_awp   

Отбор признаков логистической регрессией

In [68]:
from sklearn.linear_model import LogisticRegression
import math 
clf = LogisticRegression(random_state=0,solver='liblinear',penalty='l1').fit(X, y)
feature_imp = pd.Series(abs(clf.coef_[0]),index=dataset.iloc[:, :-1].columns).sort_values(ascending=False)

feature_imp

ct_health                       0.979703
t_health                        0.872758
ct_armor                        0.686337
t_armor                         0.632656
t_helmets                       0.482000
ct_weapon_m4a4                  0.346949
t_weapon_ak47                   0.283899
t_weapon_sg553                  0.280256
t_grenade_smokegrenade          0.243720
ct_weapon_awp                   0.237138
ct_money                        0.202035
t_weapon_glock                  0.170509
ct_grenade_incendiarygrenade    0.170499
t_money                         0.170032
t_grenade_flashbang             0.164252
ct_weapon_sg553                 0.159041
t_grenade_molotovgrenade        0.156521
ct_weapon_aug                   0.150974
t_weapon_awp                    0.139957
t_weapon_deagle                 0.135438
ct_weapon_ak47                  0.130440
t_weapon_usps                   0.118370
ct_weapon_mp9                   0.098338
ct_helmets                      0.088642
ct_weapon_usps  

Сравнение работы случайного леса с использованием кросс-валидации

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40,random_state=42,shuffle=False)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score

rfc = RandomForestClassifier(n_estimators=10,random_state=0)
print(cross_val_score(rfc, X, y, cv=5,n_jobs=-1))
rfc.fit(X_train, y_train)
print(rfc.score(X_test, y_test))



[0.73556082 0.73809329 0.73764398 0.73286496 0.72428723]
0.7289232905808349


Сравнение градиентного бустинга и леса

In [70]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,random_state=42,shuffle=False)
clf = GradientBoostingClassifier(random_state=0).fit(X_train[:, :-10], y_train)
print(clf.score(X_test[:, :-10], y_test))

0.7449963238297524
CPU times: user 20.8 s, sys: 56.7 ms, total: 20.9 s
Wall time: 20.8 s


In [71]:
%%time
rfc = RandomForestClassifier(random_state=0).fit(X_train[:, :-10], y_train)
print(rfc.score(X_test[:, :-10], y_test))

0.7427293521771097
CPU times: user 12.9 s, sys: 76 ms, total: 12.9 s
Wall time: 12.9 s
